# Programa para baixar CSVs de perfis do Twitter com API

In [1]:
# Pacotes necessários
import pandas as pd
import numpy as np
import os
import tweepy
import json
import csv
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [2]:
# Base de dados com lista de perfis de interesse
# Fonte dos dados: https://zenodo.org/deposit/5073680
os.chdir("C:/Users/andre/OneDrive/Pesquisas/ANDRE_TESE_COVID/TeseDados/TwitterCOVID_Zenodo")
dt = pd.read_csv('Brazil_Portuguese_COVID19_Tweets2020.csv')

In [3]:
# Cria lista com os nomes dos usuários
column_values = dt[['username']].values.ravel()  # dataframe com o nome dos usuários
unique_values = pd.unique(column_values)         # remove nomes duplicados
users = unique_values.tolist()                   # converte o dataframe em uma lista com os nomes

In [4]:
# Autenticação no Twitter via API
api_key = ''        # inserir chave
secret_key = ''     # inserir chave
access_token = ''   # inserir chave
secret_token = ''   # inserir chave

def authenticate(api_key, secret_key, access_token, secret_token):

    auth = tweepy.OAuthHandler(api_key, secret_key)
    auth.set_access_token(access_token, secret_token)
    api = tweepy.API(auth, 
                     wait_on_rate_limit=True, 
                     wait_on_rate_limit_notify=True)
  
    return api


api = authenticate(api_key, secret_key, access_token, secret_token)

In [6]:
# Coleta as descrições dos usuários e salva em CSV
os.chdir("C:/Users/.../Nome_da_Pasta")  # definir o caminho da pasta que ficarão salvo os arquivos
count = 0
n =           # colocar até que número deve ser o looping
while (count < n):
    count += 1
    a += 100                                                 # número do primeiro usuário a ser salvo
    b = 100+a                                                # número do último usuário a ser salvo
    csv_number = int(a/100)                                  # define o número do csv salvo
    filecsv_name = "TwitterUsersBio{}".format(csv_number)    # define o nome do csv salvo
    list_of_users = users[a:b]                               # carrega 100 observações por vez (máximo permitido)

    user_objects = api.lookup_users(screen_names=list_of_users)   # minera no Twitter dados de cada usuário
    user_cre = [user.created_at for user in user_objects]         # data que o usuário criou o perfil
    user_ids = [user.id_str for user in user_objects]             # id do usuário
    user_nam = [user.screen_name for user in user_objects]        # nome de usuário
    user_des = [user.description for user in user_objects]        # como o usuário se descreve (profissão, religião etc.)
    user_ver = [user.verified for user in user_objects]           # se é uma conta verificada pelo Twitter
    user_pro = [user.protected for user in user_objects]          # se é uma conta aberta ou fechada (protegida)
    user_fol = [user.followers_count for user in user_objects]    # número de seguidores
    user_fri = [user.friends_count for user in user_objects]      # número de amigos
    user_loc = [user.location for user in user_objects]           # localização declarada pelo usuário
    bio = pd.DataFrame(list(zip(user_cre,                         # carraga dataframe das variáveis
                            user_ids,
                            user_nam,
                            user_des,
                            user_ver,
                            user_pro,
                            user_fol,
                            user_fri,
                            user_loc)),columns = ['created_at',
                                                  'user_id',
                                                  'screen_name',
                                                  'description',
                                                  'verified',
                                                  'protected',
                                                  'nfollowers',
                                                  'nfriends',
                                                  'location'])
    bio.to_csv(filecsv_name + '.csv')                             # salva csv 

In [7]:
# Unifica em única base todos os CSVs baixados
# define as colunas de interesse
common_columns = ['created_at', 'user_id','screen_name','description','verified','protected','nfollowers','nfriends',
                  'location']

# define o tipo de objeto em cada coluna
dtypes = {'created_at': np.str, 'user_id': np.str,'screen_name': np.str,'description': np.str, 'verified': np.str,
          'protected': np.str, 'nfollowers':np.str,'nfriends': np.str, 'location': np.str}

# cria um dataframe com todos CSVs na pasta com os downloads
ddf = dd.read_csv('*.csv', dtype = dtypes, usecols = common_columns)

# transforma dataframe no formato pandas
df = ddf.compute()
df

,created_at,user_id,screen_name,description,verified,protected,nfollowers,nfriends,location
0,2019-05-23 19:09:34,1131638318435885056,PMSilvianopolis,Bem-vindo ao twitter oficial da Prefeitura Mun...,False,False,19,97,"Silvianópolis, Brasil"
1,2009-05-07 15:30:01,38451939,Rtobaldini,NaN,False,False,55,215,"Florianópolis, Brazil"
2,2013-09-02 16:23:40,1722831997,_sempregabriele,Twitter é a fila do mercado e eu sou minha mãe...,False,False,59,323,Tão tão distante...
3,2009-07-16 17:03:51,57383875,aapp_oficial,Twitter oficial da A. A. Ponte Preta. Curta a ...,True,False,201007,48,Campinas
4,2019-02-03 14:31:39,1092068099321417733,ze_prof,"Professor e botafoguense, não necessariamente ...",False,False,74,552,"Andaraí, Rio de Janeiro"
...,...,...,...,...,...,...,...,...,...
78,2013-08-24 20:20:44,1697382919,ValeskaLoureiro,"Peça, Acredite e Receba",False,False,98,91,Curitiba Paraná Brasil
79,2021-04-22 02:12:09,1385053514699784195,Catarin47107787,NaN,False,False,277,563,"Rio de Janeiro, Brasil"
80,2020-08-25 04:28:29,1298114933373243402,sonsareina,🧠 18 com cara de 15 🧠\n🌱 Amante de Brontë e Au...,False,False,64,53,"Conchal, Brasil"
81,2011-04-15 15:18:19,282617725,gerodrigot,Dia 31 de fevereiro eu vou revelar todos os me...,False,False,1427,948,CWB
